# Trabajo Práctico Nro. 1C
### Ing. Javier Ouret - 2025 - UCA - Facultad de Ingeniería
## Cliente Servidor utilizando Sockets en Python. (Entrega 19/05/2025)

Para realizar programas Cliente Servidor con Python utlizamos la librería o paquete socket.py ( https://github.com/python/cpython/blob/3.10/Lib/socket.py ).   
Esta librería es una transcripción sencilla de la llamada al sistema sockets de BSD Unix al estilo orientado a objetos de Python.   
La función socket() devuelve a socket object métodos que implementan las diversas llamadas al sistema de socket.   
Los tipos de parámetros tienen un nivel algo más alto que en la interfaz C, como con read() y write() en el uso de los archivos Python, la asignación del buffer es automática y la longitud del buffer está implícita en las operaciones de envío.

#### ¿Qué es un hilo o thread en Python?

- Es una unidad de ejecución dentro de un proceso. 
- Python permite que un proceso ejecute múltiples hilos de manera concurrente, compartiendo memoria pero realizando tareas de forma aparentemente simultánea.
- Los hilos (threads) utilizan un mecanismo de time sharing o compartición del tiempo para asignar los recursos del procesador.
- El sistema operativo asigna tiempos de ejecución (en segmentos de tiempo) a cada hilo dentro de un proceso.
- Múltiples hilos comparten el mismo recurso (el procesador) de forma simultánea o concurrente.
- Los hilos (threads) funcionan de manera similar a los procesos asincrónicos.
- Los hilos se ejecutan dentro de un solo proceso, compartiendo el mismo espacio de memoria.
- Los hilos pueden ser concurrentes o paralelos, dependiendo de si el sistema tiene un solo núcleo de CPU o múltiples núcleos.
- Los hilos se utilizan en un enfoque concurrente debido al Global Interpreter Lock (GIL).
- Aunque los hilos parecen ejecutarse al mismo tiempo, en realidad solo uno de ellos ejecuta código Python en un momento dado.
- Los hilos son útiles para tareas ligeras o I/O-bound (como esperar por datos de una base de datos, leer archivos o hacer solicitudes de red), donde la ejecución de código no consume mucha CPU.

#### Procesos Asincrónicos (usando asyncio en Python):

- Procesos asincrónicos o corutinas permiten que varias tareas se ejecuten en el mismo hilo de manera concurrente sin necesidad de crear múltiples hilos o procesos.
- Las tareas asincrónicas permiten no bloquear el hilo mientras esperan una respuesta
- Util para tareas I/O-bound, como hacer solicitudes de red o leer archivos de forma eficiente.
- Las tareas asincrónicas funcionan mediante un bucle de eventos (event loop) que gestiona las tareas que se ejecutan de manera no bloqueante.
- Cuando una tarea espera por I/O (como una solicitud HTTP), el event loop puede ejecutar otras tareas sin esperar que se complete la operación de I/O.
- Aunque asincrónico significa que las tareas no se bloquean entre sí, los procesos asincrónicos siguen siendo ejecutados en un solo hilo.
  
#### ¿Por qué usamos threads en un servidor TCP?

- Un servidor TCP básico acepta conexiones de clientes.
- Si sólo usamos un hilo (el principal), el servidor solo podría atender a un cliente a la vez.
- Para atender a múltiples clientes simultáneamente, se usan hilos concurrentes


- Cada vez que llega un cliente, se crea un nuevo hilo que ejecuta proceso_hijo(), permitiendo que el servidor siga aceptando nuevas conexiones.
- Cada hilo:
  - Maneja una conexión independiente.
  - Recibe y responde mensajes al cliente.
  - Se cierra cuando el cliente deja de enviar datos.
- A tener en cuenta las siguientes limitaciones:
  - Python usa GIL (Global Interpreter Lock), lo que limita el paralelismo en threads CPU-intensivos.
  - Para sistemas con muchas conexiones simultáneas, se recomienda usar asyncio, multiprocessing o servidores no bloqueantes con select.

#### Si usamos Flask (microframework)
- Flask se ejecuta en un solo hilo (por defecto).
- Al usar WebSockets y threading o eventlet, Flask puede mantenerse receptivo mientras espera mensajes del servidor TCP.
- Con Flask, cada petición HTTP puede iniciar un hilo que se conecta al servidor o ejecuta una tarea SSH (como iniciar el servidor remoto).

- El módulo threading en Python no usa fork() internamente.

#### Diferencias entre threading y fork()
- threading:
  - Los hilos (threads) comparten el mismo espacio de memoria.
  - Los hilos son ligeros y más eficientes en términos de uso de memoria.
  - Los hilos se ejecutan dentro del mismo proceso y no crean un nuevo proceso de sistema operativo.
  - El módulo threading maneja la creación de hilos dentro del mismo proceso sin la necesidad de un sistema de procesos separados.


- fork():
  - fork() es una llamada al sistema que crea un nuevo proceso.
  - Este proceso hijo tiene su propio espacio de memoria independiente del proceso padre, pero inicialmente, el espacio de memoria es copiado del proceso padre.
  - Los procesos creados con fork() no comparten memoria, lo que los hace más costosos en términos de recursos que los hilos.

#### Nota Importante:
- Si se guardan los archivos de código con %%writefile no se ejecutan desde la celda, hay que abrir una terminal y ejecutarlos desde ahí.
- **Como son programas cliente-servidor ejecutarlos desde terminales para ver cada paso**. Con -u se ven todos los detalles de ejecución. Si se ejecutan desde el Jupyter no se ven los resultados salvo que los llame en segundo plano con & y luego haga un tail.
 - Otra opción es usar Jupyter Lab que permite ventanas en pestañas.
 - Instalarlo desde entorno virtual venv con:

#### Socket_Cliente_Connect_01.py

In [12]:
%%writefile Socket_Cliente_Connect_01.py
#!/usr/bin/python
# Socket_Cliente_Connect_01C.py
import socket
import sys
import time

# Creando un socket TCP/IP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conecta el socket en el puerto cuando el servidor esté escuchando
server_dir = ('localhost', 6667)
print('conectando a %s puerto %s' % server_dir)
sock.connect(server_dir)

contador = 0
while contador < 10:  # 10 mensajes
    # Enviando datos
    print("Paso:", contador)
    mensaje = b'123456789'
    print('Enviando mensaje "%s"' % mensaje)
    sock.sendall(mensaje)

    # Esperando la respuesta del servidor
    data = sock.recv(1024)  # Leemos hasta 1024 bytes
    if data:
        print('Recibiendo mensaje "%s"' % data.decode('utf-8'))
    else:
        print("El servidor cerró la conexión.")
        break  # Si no recibimos datos, terminamos
    
    # Pausa de 2 segundos entre cada mensaje
    time.sleep(2)  # Retraso de 1 segundo entre envíos de mensajes

    contador += 1
    print("Paso:", contador)

print('cerrando socket')
sock.close()
sys.exit(0)  # Terminar el programa


Overwriting Socket_Cliente_Connect_01.py


In [13]:
%%writefile Socket_Cliente_Connect_01B.py
import socket

HOST = '127.0.0.1'   # Cambiar IP si el servidor está en otra máquina
PORT = 6667          # Puerto del servidor

try:
    print("Paso 0: Creando socket...")
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(5)  # Tiempo máximo de espera para recibir datos

    print(f"Paso 1: Conectando a servidor en {HOST}:{PORT}...")
    sock.connect((HOST, PORT))

    bienvenida = sock.recv(1024)
    print("Paso 2: Mensaje del servidor:", bienvenida.decode('utf-8'))

    print("Paso 3: Listo para ingresar mensaje.")
    mensaje = input("Ingrese un mensaje para enviar al servidor: ")
    sock.sendall(mensaje.encode('utf-8'))
    print("Paso 4: Mensaje enviado, esperando respuesta...")

    try:
        respuesta = sock.recv(1024)
        print("Paso 5: Respuesta del servidor:", respuesta.decode('utf-8'))
    except socket.timeout:
        print("Tiempo de espera agotado. El servidor no respondió.")

except ConnectionRefusedError:
    print("No se pudo conectar al servidor. Verifique si está activo.")
except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    sock.close()
    print("Paso 6: Socket cerrado. Cliente finalizado.")



Overwriting Socket_Cliente_Connect_01B.py


Ejecutarlo con: 

Agregar -u para ver detalles

#### Socket_Cliente_Mostrar_Tupla.py

In [3]:
%%writefile Socket_Cliente_Mostrar_Tupla.py
#!/usr/bin/python
# Socket_Cliente_Mostrar_Tupla.py
import socket
import sys

# Creando un socket TCP/IP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#addr = ('0.0.0.0', 0)
#sock.bind(addr) si fuese en servidor

# Muestro tupla antes de la asignación de IP y puerto
print('Datos de la conexión antes de completar datos en estructura:', sock)


# Conecta el socket en el puerto cuando el servidor esté escuchando
server_dir = ('localhost', 6667)
print ('conectando a %s puerto %s',server_dir)
sock.connect(server_dir)
# Muestro tupla luego de la asignación de IP y puerto
print('Datos de la conexión luego de completar datos en estructura', sock)
# Datos de la tupla

hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)
print(f"Host: {hostname}")
print(f"IP: {ip_address}")

contador = 0
while contador <= 5:
    # Enviando datos
    print("Paso:", contador)
    datos = sock.recv(10)
    print(datos.decode('utf-8'))
    mensaje = b'123456789012345678901234567890'
    print('enviando "%rb"', mensaje)
    sock.sendall(mensaje)

    # Buscando respuesta
    bytes_recibidos = 0
    bytes_esperados = len(mensaje)

    while bytes_recibidos < bytes_esperados:
        data = sock.recv(10)
        bytes_recibidos += len(data)
        print('recibiendo "%s"', data)

    contador = contador + 1
    print ("Paso:", contador)

print ('cerrando socket')
sock.close()

Writing Socket_Cliente_Mostrar_Tupla.py


#### Socket_Cliente_Select.py

In [ ]:
%%writefile Socket_Cliente_Select.py
# Socket_Cliente_Select.py
# El programa cliente de ejemplo utiliza dos sockets para demostrar cómo el servidor con select() administra múltiples conexiones al mismo tiempo.
# El cliente comienza conectando cada socket TCP/IP al servidor.

import socket
import sys

mensajes = [
    'Este mensaje ',
    'es enviado ',
    'en partes.',
]
dir_servidor = ('localhost', 10000)

# Creo socket
socks = [
    socket.socket(socket.AF_INET, socket.SOCK_STREAM),
    socket.socket(socket.AF_INET, socket.SOCK_STREAM),
]

# Conectar el socket al puerto en el cual el servidor está escuchando
print('conectando a {} puerto {}'.format(*dir_servidor),
      file=sys.stderr)
for s in socks:
    s.connect(dir_servidor)
for mensaje in mensajes:
    datos_salientes = mensaje.encode()

    # envío mensajes en ambos sockets
    for s in socks:
        print('{}: enviando {!r}'.format(s.getsockname(),
                                        datos_salientes),
              file=sys.stderr)
        s.send(datos_salientes)

    # leo respuestas en ambos sockets
    for s in socks:
        data = s.recv(1024)
        print('{}: recibido {!r}'.format(s.getsockname(),
                                         data),
              file=sys.stderr)
        if not data:
            print('cerrando socket', s.getsockname(),
                  file=sys.stderr)
            s.close()

#### Socket_Cliente_Select_Lento.py
Esta versión «lenta» del programa cliente se detiene después de enviar cada mensaje,
para simular la latencia u otro retraso en la transmisión.

In [4]:
%%writefile Socket_Cliente_Select_Lento.py
# Socket_Cliente_Select_Lento.py

import socket
import sys
import time

# Crear un socket TCP/IP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conectar el socket al puerto en el cual el servidor está escuchando
dir_servidor = ('localhost', 10000)
print('conectando a {} puerto {}'.format(*dir_servidor),
      file=sys.stderr)
sock.connect(dir_servidor)

time.sleep(1)

mensajes = [
    'Parte 1 del mensaje',
    'Parte 2 del mensaje',
]
datos_esperados = len(''.join(mensajes))

try:

    # enviando datos
    for mensaje in mensajes:
        data = mensaje.encode()
        print('enviando {!r}'.format(data), file=sys.stderr)
        sock.sendall(data)
        time.sleep(2.5)

    # Respuesta
    datos_recibidos = 0

    while datos_recibidos < datos_esperados:
        data = sock.recv(16)
        datos_recibidos += len(data)
        print('recibidos {!r}'.format(data), file=sys.stderr)

finally:
    print('cerrando socket', file=sys.stderr)
    sock.close()

Overwriting Socket_Cliente_Select_Lento.py


#### Socket_Servidor_Concurrente_01.py

#### Socket_Servidor_Select.py
- El módulo select proporciona acceso a funciones específicas de E/S.
- La función POSIX select(), está disponible en Unix y Windows.
- El módulo también incluye poll(), pero solo para Unix, y varias opciones que solo funcionan con variantes específicas de Unix.
- El ejemplo del servidor de eco lo mejoramos para tener más de una conexión simultánea.
- La nueva versión comienza creando un socket TCP/IP que no se bloquea y configurado para escuchar en una dirección.
- Los argumentos para select() son tres listas que contienen canales de comunicación a monitorear.
  - 1. Lista de los objetos para verificar los datos entrantes que se leerán.
  - 2. Lista de objetos que recibirán datos salientes cuando haya espacio en el buffer.
  - 3. Lista de objetoss que pueden tener un error (generalmente un combinación de los objetos del canal de entrada y salida).
- El siguiente paso en el servidor es configurar las listas que contienen fuentes de entrada y los destinos de salida que se pasarán a select().
- El bucle principal del servidor agrega y elimina las conexiones de estas listas.
- Dado que esta versión del servidor va a esperar poder escribir a un socket antes de enviar cualquier dato (en lugar de enviar inmediatamente la respuesta), cada conexión de salida necesita una cola para actuar como un buffer para los datos que se enviarán a través de él.
- La parte del programa principal del servidor hace un bucle, llamando a select() para bloquear y esperar la actividad de la red.
- select() devuelve tres nuevas listas, que contienen subconjuntos del contenido de las listas pasadas. 
  - 1. Sockets lista readable tiene datos entrantes almacenados en búfer y disponibles para ser leídos.
  - 2. Sockets lista writable tienen espacio libre en su búfer y se puede escribir en ellos. 
  - 3. Sockets lista excepcional han tenido un error (la definición real de «condición excepcional» depende de la plataforma).
- Los sockets «legibles» representan tres casos posibles. 
  - Si el socket es el socket principal del «servidor», el que se usa para escuchar conexiones, entonces la condición «legible» significa que está listo para aceptar otra conexión entrante. Además de añadir la nueva conexión a la lista de entradas para monitorear, esta sección establece que el socket del cliente no se bloquee.
  - Si es una conexión establecida con un cliente que ha enviado datos, los datos se leen con recv(), luego se colocan en la cola para que puedan ser enviados a través del socket y de vuelta al cliente. Si hay datos en la cola para una conexión, se envía el siguiente mensaje.
  - Caso contrario, la conexión se elimina de la lista de conexiones de salida para que la próxima vez a través del bucle select() no indique que el socket está listo para enviar datos.
- select() puede tener un cuarto parámetro opcional, que es el número de segundos a esperar antes de interrumpir el monitoreo si no se han activado canales. El uso de un valor de tiempo de espera permite a un programa principal ejecutar select() como parte de un ciclo de procesamiento más grande,tomando otras acciones entre la comprobación de entrada de red.
  - Cuando el tiempo de espera expira, select() devuelve tres listas vacías.
  - Actualizar el ejemplo del servidor para usar un tiempo de espera requiere agregar el argumento extra a la llamada select() y manejo de las listas vacías que select() devuelve.

#### Referencias : Python-3 - Doug Hellmann

In [1]:
%%writefile Socket_Servidor_Concurrente_01.py
# Socket_Servidor_Concurrente_01.py
#servidor concurrente
import socket
import threading

host = "127.0.0.1"
port = 6667

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print ("Socket Creado")
sock.bind((host, port))
print ("socket bind Completado")
sock.listen(1)
print ("socket en modo escucha - pasivo")

def proceso_hijo(*args): #*args valores de conexión y dirección de cliente devueltos por sock.accept()
    conn = args[0] #Conexión
    addr = args[1] #Dir Cliente
    try:
        print('conexion con {}.'.format(addr))
        conn.send("Servidor: Conectado con cliente".encode('UTF-8'))

        while True:
            data = conn.recv(10)
            print ('recibido "%s"',data)
            if data:
                print ('enviando mensaje de vuelta al cliente')
                conn.sendall(data)
            else:
                print ('no hay mas datos', addr)
                break

        # while True:
        #
        #     datos = conn.recv(4096)
        #     if datos:
        #         print('Recibido: {}'.format(datos.decode('utf-8')))
        #
        #     else:
        #         print("Prueba")
        #         break
    finally:
        conn.close()

while 1:
    conn, addr = sock.accept()
    threading.Thread(target=proceso_hijo, args=(conn, addr)).start()

Overwriting Socket_Servidor_Concurrente_01.py


In [2]:
%%writefile Socket_Servidor_Concurrente_02.py
# Socket_Servidor_Concurrente_01.py
# servidor concurrente con opción de apagado

import socket
import threading
import time

host = "127.0.0.1"
port = 6667

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind((host, port))
print("Socket bind completado")
sock.listen(5)
print("Socket en modo escucha - pasivo")

clientes = []
apagado = threading.Event()

def proceso_hijo(conn, addr):
    print('Conexión con {}.'.format(addr))
    clientes.append(conn)
    try:
        conn.send("Servidor: Conectado con cliente\n".encode('utf-8'))

        while not apagado.is_set():
            data = conn.recv(10)
            if data:
                print('Recibido de {}: {}'.format(addr, data.decode('utf-8')))
                conn.sendall(data)
            else:
                print('No hay más datos de', addr)
                break
    except Exception as e:
        print("Error con {}: {}".format(addr, e))
    finally:
        conn.close()
        clientes.remove(conn)

def control_apagado():
    while not apagado.is_set():
        time.sleep(30)
        respuesta = input("\n¿Desea apagar el servidor? (s/n): ").strip().lower()
        if respuesta == 's':
            apagado.set()
            print("Apagando servidor...")
            break

# Iniciar el hilo de control
threading.Thread(target=control_apagado, daemon=True).start()

# Aceptar conexiones mientras no se apague
try:
    while not apagado.is_set():
        sock.settimeout(1.0)
        try:
            conn, addr = sock.accept()
            threading.Thread(target=proceso_hijo, args=(conn, addr)).start()
        except socket.timeout:
            continue
finally:
    print("Cerrando todas las conexiones...")
    for c in clientes:
        c.close()
    sock.close()
    print("Servidor finalizado.")


Writing Socket_Servidor_Concurrente_02.py


#### Socket_Servidor_Select.py

In [3]:
%%writefile Socket_Servidor_Select.py
# Socket_Servidor_Select.py
# Servidor con Select()

import select
import socket
import sys
import queue

# Creando un socket TCP/IP 
servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
servidor.setblocking(0)

# Hago Bind del socket al puerto
dir_servidor = ('localhost', 10000)
print('iniciando en {} port {}'.format(*dir_servidor),
      file=sys.stderr)
servidor.bind(dir_servidor)

# Escucho conexiones entrantes
servidor.listen(5)

# Sockect que espero leer
entradas = [servidor]

# Sockets que espero enviar
salidas = []

# Cola de mensajes salientes
cola_mensajes = {}

while entradas:

    # Espero a que al menos uno de los sockets este listo para ser procesado
    
    print('esperando el próximo evento', file=sys.stderr)
    readable, writable, exceptional = select.select(entradas,
                                                    salidas,
                                                    entradas)

    if not (readable or writable or exceptional):
        print('  tiempo excedido....',
              file=sys.stderr)
        continue
    # Manejo entradas
    for s in readable:

        if s is servidor:
            # Un socket "leíble" está listo para aceptar conexiones
            con, dir_cliente = s.accept()
            print('  conexión desde: ', dir_cliente,
                  file=sys.stderr)
            con.setblocking(0)
            entradas.append(con)

            # Le asigno a la conexión una cola en la cuál quiero enviar
            cola_mensajes[con] = queue.Queue()

        else:
            data = s.recv(1024)
            if data:
                # Un socket leíble tiene datos
                print('  recibido {!r} desde {}'.format(
                    data, s.getpeername()), file=sys.stderr,
                )
                cola_mensajes[s].put(data)
                # Agrego un canal de salida para la respuesta
                if s not in salidas:
                    salidas.append(s)
            else:
                # Si está vacío lo interpreto como una conexión a cerrar
                print('  cerrando...', dir_cliente,
                      file=sys.stderr)
                # dejo de escuchar en la conexión
                if s in salidas:
                    salidas.remove(s)
                entradas.remove(s)
                s.close()

                # Rremueve mensaje de la cola
                del cola_mensajes[s]
    # Administro salidas
    for s in writable:
        try:
            next_msg = cola_mensajes[s].get_nowait()
        except queue.Empty:
            # No hay mensaje en espera. Dejo de controlar para posibles escrituras
            
            print('  ', s.getpeername(), 'cola vacía',
                  file=sys.stderr)
            salidas.remove(s)
        else:
            print(' enviando {!r} a {}'.format(next_msg, s.getpeername()), file=sys.stderr)
            s.send(next_msg)

  # Administro condiciones excepcionales"
    for s in exceptional:
        print('excepción en', s.getpeername(),
              file=sys.stderr)
        # Dejo de escuchar en las conexiones
        entradas.remove(s)
        if s in salidas:
            salidas.remove(s)
        s.close()

        # Remuevo cola de mensajes
        del cola_mensajes[s]


Overwriting Socket_Servidor_Select.py


### Versión con rutas
servidor_con_interfaz.py
Observar que es necesario refrescar el navegador para ver los mensajes.

In [20]:
%%writefile servidor_con_interfaz.py
from flask import Flask, render_template_string, request, redirect, url_for
import socket
import threading
import time

app = Flask(__name__)
clientes = []
mensajes = []
clientes_lock = threading.Lock()
mensajes_lock = threading.Lock()
apagado = threading.Event()

HOST = '127.0.0.1'
PORT = 6667

# ========================
# FLASK - INTERFAZ WEB
# ========================
@app.route('/')
def index():
    with clientes_lock:
        lista_clientes = list(clientes)
    with mensajes_lock:
        log = list(mensajes)
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
        <meta http-equiv="refresh" content="3">
        <title>Servidor TCP</title>
    </head>
    <body>
    <h2>Servidor TCP</h2>
    <p><b>Clientes conectados:</b></p>
    <ul>
      {% for c in clientes %}
        <li>{{ c }}</li>
      {% else %}
        <li>No hay clientes conectados</li>
      {% endfor %}
    </ul>
    <p><b>Mensajes recibidos:</b></p>
    <ul>
      {% for m in mensajes %}
        <li>{{ m }}</li>
      {% else %}
        <li>No hay mensajes</li>
      {% endfor %}
    </ul>
    <form method="post" action="/apagar">
        <button type="submit">Apagar servidor</button>
    </form>
    </body>
    </html>
    """, clientes=lista_clientes, mensajes=log)
    
@app.route('/apagar', methods=['POST'])
def apagar():
    apagado.set()
    return redirect(url_for('index'))

# ========================
# SERVIDOR TCP
# ========================
def proceso_hijo(conn, addr):
    with clientes_lock:
        clientes.append(f"{addr[0]}:{addr[1]}")
    try:
        conn.send("Servidor: Conectado con cliente\n".encode('utf-8'))
        while not apagado.is_set():
            data = conn.recv(1024)
            if data:
                mensaje = f"De {addr[0]}:{addr[1]} → {data.decode('utf-8').strip()}"
                print(mensaje)
                with mensajes_lock:
                    mensajes.append(mensaje)
                    if len(mensajes) > 20:
                        mensajes.pop(0)
                conn.sendall(data)
            else:
                break
    except Exception as e:
        print(f"Error con {addr}: {e}")
    finally:
        conn.close()
        with clientes_lock:
            clientes.remove(f"{addr[0]}:{addr[1]}")

def servidor_tcp():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind((HOST, PORT))
    sock.listen(5)
    sock.settimeout(1.0)
    print("Servidor TCP escuchando en", (HOST, PORT))
    try:
        while not apagado.is_set():
            try:
                conn, addr = sock.accept()
                threading.Thread(target=proceso_hijo, args=(conn, addr)).start()
            except socket.timeout:
                continue
    finally:
        sock.close()
        print("Servidor apagado.")

# ========================
# MAIN
# ========================
if __name__ == "__main__":
    # Inicia servidor TCP en un hilo
    threading.Thread(target=servidor_tcp, daemon=True).start()

    # Ejecuta la app Flask
    app.run(debug=False, port=5000)


Overwriting servidor_con_interfaz.py


### Versión con rutas RestAPI
servidor_con_interfaz_ReestAPI.py
Requiere otro tipo de cliente, por qué ?

In [7]:
%%writefile cliente_restapi.py
# Cliente usando requests para comunicarse con Flask (simulando el envío de mensajes 10 veces)
#!/usr/bin/python
# cliente_restapi.py

import requests
import time
import sys

# Dirección del servidor Flask
URL = "http://localhost:5000"

contador = 0
while contador < 10:
    print(f"Paso: {contador}")

    # Simulamos un mensaje que el cliente quiere enviar al servidor
    mensaje = f"Mensaje {contador}"
    print(f"Enviando mensaje: {mensaje}")

    try:
        # Enviar el mensaje al servidor a través de una solicitud POST
        response = requests.post(f"{URL}/mensaje", json={"mensaje": mensaje})
        if response.status_code == 200:
            print("Respuesta del servidor:", response.json().get("respuesta"))
        else:
            print("Error del servidor:", response.status_code)
    except Exception as e:
        print("Error en la conexión:", e)
        break

    time.sleep(2)  # Esperar 2 segundos
    contador += 1

print("Finalizando cliente.")
sys.exit(0)

Overwriting cliente_restapi.py


In [9]:
%%writefile servidor_con_interfaz_RestAPI.py
from flask import Flask, jsonify, render_template, request, redirect, url_for
import time
import threading

app = Flask(__name__)

clientes = []
mensajes = []
clientes_lock = threading.Lock()
mensajes_lock = threading.Lock()
apagado = threading.Event()

# ========================
# FLASK - RUTAS API
# ========================
@app.route('/')
def index():
    with clientes_lock:
        lista_clientes = list(clientes)
    with mensajes_lock:
        lista_mensajes = list(mensajes)
    return render_template("index.html", clientes=lista_clientes, mensajes=lista_mensajes)

@app.route('/mensaje', methods=['POST'])
def recibir_mensaje():
    data = request.get_json()
    mensaje = data.get("mensaje", "")
    ip_cliente = request.remote_addr
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

    with clientes_lock:
        existe = any(c['ip'] == ip_cliente for c in clientes)
        if not existe:
            clientes.append({"ip": ip_cliente, "conexion_time": timestamp})

    with mensajes_lock:
        mensajes.append(f"{timestamp} - {ip_cliente}: {mensaje}")
        if len(mensajes) > 50:
            mensajes.pop(0)

    print(f"[Servidor] {ip_cliente} → {mensaje}")
    return jsonify({"respuesta": f"Mensaje recibido: {mensaje}"}), 200

@app.route('/clientes', methods=['GET'])
def obtener_clientes():
    with clientes_lock:
        return jsonify(clientes)

@app.route('/apagar', methods=['POST'])
def apagar():
    apagado.set()
    return redirect(url_for('index'))

# ========================
# SERVIDOR EN HILO
# ========================
def servidor_loop():
    while not apagado.is_set():
        time.sleep(1)
    print("Servidor apagado.")

# ========================
# MAIN
# ========================
if __name__ == "__main__":
    threading.Thread(target=servidor_loop, daemon=True).start()
    app.run(debug=True, port=5000)



Overwriting servidor_con_interfaz_RestAPI.py


In [11]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Servidor REST - Clientes y Mensajes</title>
    <meta http-equiv="refresh" content="5">  <!-- Refrescar cada 5 segundos -->
    <style>
        body { font-family: Arial; background: #f2f2f2; padding: 20px; }
        h1 { color: #333; }
        .panel { background: white; padding: 20px; border-radius: 5px; margin-bottom: 20px; box-shadow: 0 0 10px #ccc; }
        ul { list-style-type: none; padding: 0; }
        li { margin-bottom: 5px; }
        button { padding: 10px 20px; background: crimson; color: white; border: none; border-radius: 4px; cursor: pointer; }
        button:hover { background: darkred; }
    </style>
</head>
<body>
    <h1>Servidor REST API</h1>

    <div class="panel">
        <h2>Clientes conectados</h2>
        <ul>
            {% if clientes %}
                {% for c in clientes %}
                    <li>{{ c.ip }} — desde {{ c.conexion_time }}</li>
                {% endfor %}
            {% else %}
                <li>No hay clientes conectados</li>
            {% endif %}
        </ul>
    </div>

    <div class="panel">
        <h2>Mensajes recibidos</h2>
        <ul>
            {% if mensajes %}
                {% for m in mensajes %}
                    <li>{{ m }}</li>
                {% endfor %}
            {% else %}
                <li>No hay mensajes</li>
            {% endif %}
        </ul>
    </div>

    <form method="post" action="/apagar">
        <button type="submit">Apagar servidor</button>
    </form>
</body>
</html>



Overwriting templates/index.html


In [6]:
%%writefile static/css/style.css
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f9;
    margin: 0;
    padding: 0;
}

.container {
    width: 80%;
    margin: auto;
    background-color: white;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
    margin-top: 50px;
}

h2 {
    text-align: center;
    color: #333;
}

h3 {
    color: #444;
}

table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 20px;
}

th, td {
    padding: 10px;
    text-align: left;
    border-bottom: 1px solid #ddd;
}

th {
    background-color: #f7f7f7;
}

button {
    padding: 10px 20px;
    border: none;
    background-color: #4CAF50;
    color: white;
    cursor: pointer;
    border-radius: 5px;
    margin: 5px;
    transition: background-color 0.3s;
}

button:hover {
    background-color: #45a049;
}

.apagar-btn {
    background-color: #f44336;
}

.apagar-btn:hover {
    background-color: #e53935;
}


Overwriting static/css/style.css


Acceder con: http://127.0.0.1:5000

### Consigna del TP 1C
**Ejercicios a realizar durante la clase. Incluir los códigos dentro de este mismo Notebook**

- Por qué es necesario refrescar el servidor_con_interfaz.py para poder ver los mensajes?
- Funciona apagar servidor ? Cómo funciona ? Qué hace ?
- Escribir una aplicación cliente servidor en python que:
  - Envíe un archivo pequeño.
  - Que muestre las direcciones y puertos de todos los clientes conectados del lado del servidor.
  - Que devuelva a cada cliente el día y hora de conexión, y el tiempo que estuvo (o está) conectado.   
  <br>

- Realizar la misma aplicación tanto para C-S con Concurrencia Aparente (Select) como C-S Concurrente.

- De ser necesario agregar tiempo de espera, loop, sleep, con contadores para demorar los procesos.

- Implementar una limpieza de recuersos al salir del los programas (agregar opción de pregunta al usuario para cerrar los clientes).

Para crear un socket (stream) en Python: socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)
Los parámetros son los mismos que se usan en C.